In [1]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import pyterrier as pt

import requests
import json

if not pt.started():
    pt.init()

nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")

C:\Users\Mohammad Kher\AppData\Local\Temp\ipykernel_6428\3734961694.py:12: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():
Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]
C:\Users\Mohammad Kher\AppData\Local\Temp\ipykernel_6428\3734961694.py:13: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()
[nltk_data] Downloading package punkt to C:\Users\Mohammad
[nltk_data]     Kher\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\Mohammad
[nltk_data]     Kher\AppData\Roaming\nltk_data...
[nltk_data]   Packa

True

#### Get Lawyers Data form API

In [2]:
lawyer_response = requests.get("http://osamanaser806-32078.portmap.io:32078/api/v1/ai/lawyers")
f"status code : {lawyer_response.status_code}"

'status code : 200'

In [31]:
lawyers_response = lawyer_response.json()
lawyers = pd.DataFrame(lawyers_response['lawyers']).rename(columns={
    "id":"lawyer_id"
}).drop(["email","union_number","affiliation_date","phone","rates","rank","avatar"],axis=1)
lawyers.head()

,lawyer_id,name,address,union_branch,specializations,years_of_experience,description
0,1,هديل الزهر,ريف دمشق - دوما,دمشق,"[Civil, Criminal, Arbitration, Legal]",5,قلمكم موقع متخصص بتقديم خدمة الترجمة المعتمدة،...
1,2,عبدالرزاق شاكر اسيد الخضيري,79 ممر ياسمين السماعيل عمارة رقم 42\nبيشة,جيزان,"[Military, Arbitration, Commercial]",17,Qui velit dolor consequatur. Optio facilis tem...
2,3,رماح الجريد,25244 ممر مريم هوساوي\nوادي الدواسر,ينبع البحر,"[Administrative, Civil, Commercial, Arbitration]",29,Qui odit autem quibusdam voluptatem dolor. Eaq...
3,4,شوان مهند الفدا,9006 شارع معاذ العتيبي\nالطائف,نجران,"[Administrative, Terrorism, Commercial, Civil,...",38,Dolor soluta esse sunt repellat qui nobis reru...
4,5,مازن طه الفريدي,64 شارع سرين سقا\nخميس مشيط,الباحة,"[Civil, Terrorism, International Disputes, Leg...",17,Sunt sequi in et doloremque qui nemo expedita ...


In [33]:
lawyers['years_of_experience'] = lawyers['years_of_experience'].apply(lambda x: f"{x}year")
# lawyers['rank'] = lawyers['rank'].apply(lambda x: f"{int(x)}start")
lawyers.head()

,lawyer_id,name,address,union_branch,specializations,years_of_experience,description
0,1,هديل الزهر,ريف دمشق - دوما,دمشق,"[Civil, Criminal, Arbitration, Legal]",5yearyear,قلمكم موقع متخصص بتقديم خدمة الترجمة المعتمدة،...
1,2,عبدالرزاق شاكر اسيد الخضيري,79 ممر ياسمين السماعيل عمارة رقم 42\nبيشة,جيزان,"[Military, Arbitration, Commercial]",17yearyear,Qui velit dolor consequatur. Optio facilis tem...
2,3,رماح الجريد,25244 ممر مريم هوساوي\nوادي الدواسر,ينبع البحر,"[Administrative, Civil, Commercial, Arbitration]",29yearyear,Qui odit autem quibusdam voluptatem dolor. Eaq...
3,4,شوان مهند الفدا,9006 شارع معاذ العتيبي\nالطائف,نجران,"[Administrative, Terrorism, Commercial, Civil,...",38yearyear,Dolor soluta esse sunt repellat qui nobis reru...
4,5,مازن طه الفريدي,64 شارع سرين سقا\nخميس مشيط,الباحة,"[Civil, Terrorism, International Disputes, Leg...",17yearyear,Sunt sequi in et doloremque qui nemo expedita ...


### Get Rate Data Form API

In [34]:
rate_response = requests.get("http://osamanaser806-32078.portmap.io:32078/api/v1/ai/rates")
f"status code : {rate_response.status_code}"

'status code : 200'

In [35]:
rates_response = rate_response.json()
rates = pd.DataFrame(rates_response["rates"])
rates.dropna(inplace=True)
rates.drop(["id"],axis=1,inplace=True)
rates.head()

,user_id,lawyer_id,rating,review
0,33,2,3,Ut dignissimos molestiae quos praesentium. Qui...
1,32,16,5,Commodi in voluptate dolorem aperiam impedit n...
2,36,48,4,Et minima repellat et quia odio rerum. Commodi...
3,15,13,1,Ea asperiores animi totam numquam et. Tempore ...
4,50,27,5,Error excepturi quia natus aliquid non et. Und...


In [36]:
rates['rating'] = rates['rating'].apply(lambda x :f"{x}star")
rates.head()

,user_id,lawyer_id,rating,review
0,33,2,3star,Ut dignissimos molestiae quos praesentium. Qui...
1,32,16,5star,Commodi in voluptate dolorem aperiam impedit n...
2,36,48,4star,Et minima repellat et quia odio rerum. Commodi...
3,15,13,1star,Ea asperiores animi totam numquam et. Tempore ...
4,50,27,5star,Error excepturi quia natus aliquid non et. Und...


### Get Agencies Data From API

In [37]:
agencie_response = requests.get("http://osamanaser806-32078.portmap.io:32078/api/v1/ai/agencies")
f"status code : {agencie_response.status_code}"

'status code : 200'

In [38]:
agencies_response = agencie_response.json()
agencies = pd.DataFrame(agencies_response["agencies"]).rename(columns={
    "id":"agency_id"
})
agencies = agencies[["agency_id","lawyer_id"]]
agencies.head()

,agency_id,lawyer_id
0,1,1
1,2,21
2,3,48
3,4,47
4,5,1


### Get Issues Data From API

In [39]:
issue_response = requests.get("http://osamanaser806-32078.portmap.io:32078/api/v1/ai/issues")
f"status code : {issue_response.status_code}"

'status code : 200'

In [84]:
issues_response = issue_response.json()
issues = pd.DataFrame(issues_response["issues"]).drop(["base_number","record_number","court_id","court_room_id"],axis=1)
issues.head()

,id,agency_id,start_date,end_date,status,estimated_cost,is_active
0,1,1,2024-10-20,None,قيد الدعوى,1500000,Yes
1,2,85,1990-01-13,2025-01-09,Atque beatae aut illo.,11672000,No
2,3,83,1990-11-06,None,Et adipisci aut est id.,2765228,Yes
3,4,99,1990-05-06,2025-01-09,Soluta id et unde enim.,8141773,No
4,5,71,1990-08-19,None,Vero in perferendis qui.,6150534,Yes


In [85]:
issues["estimated_cost"] = issues["estimated_cost"].apply(lambda x: f"{x}$")
issues.head()

,id,agency_id,start_date,end_date,status,estimated_cost,is_active
0,1,1,2024-10-20,None,قيد الدعوى,1500000$,Yes
1,2,85,1990-01-13,2025-01-09,Atque beatae aut illo.,11672000$,No
2,3,83,1990-11-06,None,Et adipisci aut est id.,2765228$,Yes
3,4,99,1990-05-06,2025-01-09,Soluta id et unde enim.,8141773$,No
4,5,71,1990-08-19,None,Vero in perferendis qui.,6150534$,Yes


In [48]:
class Preprocessing_EN:
  @staticmethod
  def process(sentance):
    sentance = Preprocessing_EN.remove_punctuation(sentance)
    sentance = Preprocessing_EN.tokenizer(sentance)
    sentance = Preprocessing_EN.normalizer(sentance)
    sentance = Preprocessing_EN.remove_stopwords(sentance)
    sentance = Preprocessing_EN.remove_deplicate(sentance)
    sentance = Preprocessing_EN.stemmer(sentance)
    return sentance

  @staticmethod
  def tokenizer(sentance):
    words = word_tokenize(sentance)
    return words

  @staticmethod
  def normalizer(sentance):
    return [word.lower() for word in sentance]

  @staticmethod
  def remove_stopwords(sentance):
    stop_words = set(stopwords.words('english'))
    sentance = [word for word in sentance if word not in stop_words]
    return sentance
  
  @staticmethod
  def stemmer(sentance):
    stemmer = PorterStemmer()
    sentance = [stemmer.stem(word) for word in sentance]
    return sentance

  @staticmethod
  def remove_punctuation(sentance):
    return re.sub(r'[^A-Za-z0-9\s]',' ',sentance)

  @staticmethod
  def remove_deplicate(sentance):
    return list(set(sentance))

In [49]:
class Preprocessing_AR:
  @staticmethod
  def process(sentance):
    sentance = Preprocessing_AR.remove_punctuation(sentance)
    sentance = Preprocessing_AR.normalizer(sentance)
    sentance = Preprocessing_AR.tokenizer(sentance)
    sentance = Preprocessing_AR.remove_stopwords(sentance)
    sentance = Preprocessing_AR.remove_deplicate(sentance)
    sentance = Preprocessing_AR.stemmer(sentance)
    
    return sentance

  @staticmethod
  def tokenizer(sentance):
    words = word_tokenize(sentance)
    return words

  @staticmethod
  def normalizer(sentance):
    sentance = re.sub(r"[إأٱآا]","ا",sentance)
    sentance = re.sub(r"ي","ى",sentance)
    sentance = re.sub(r"ء","ؤ",sentance)
    sentance = re.sub(r"ء","ئ",sentance)
    sentance = re.sub(r"[^ا-ي]"," ",sentance)
    sentance = re.sub(r"ة","ه",sentance)
    return sentance


  @staticmethod
  def remove_stopwords(sentance):
    
    stop_words = set(stopwords.words('arabic'))

    sentance = [word for word in sentance if word not in stop_words]
    return sentance

  # @staticmethod
  # def lemmatizer(sentance):
  #   farasa_tagget = FarasaPOSTagger(interaction=True)
  #   return sentance
  
  @staticmethod
  def stemmer(sentance):
    stemmer = PorterStemmer()
    sentance = [stemmer.stem(word) for word in sentance]
    return sentance

  @staticmethod
  def remove_punctuation(sentance):
    return re.sub(r'[^ا-ي 0-9\s]',' ',sentance)

  @staticmethod
  def remove_deplicate(sentance):
    return list(set(sentance))

In [50]:
def split_by_language(text):
  arabic= " ".join(re.findall(r"[\u0600-\u06FF]+", text))
  english = " ".join(re.findall(r"[a-zA-Z0-9]+", text))
  clear_text  = Preprocessing_AR.process(arabic)+Preprocessing_EN.process(english)
  return clear_text

In [51]:
lawyers_with_rates = lawyers.merge(rates,on="lawyer_id")
lawyers_with_rates.head()

,lawyer_id,name,address,union_branch,specializations,years_of_experience,description,user_id,rating,review
0,1,هديل الزهر,ريف دمشق - دوما,دمشق,"[Civil, Criminal, Arbitration, Legal]",5yearyear,قلمكم موقع متخصص بتقديم خدمة الترجمة المعتمدة،...,3,5star,Inventore harum beatae aut esse consequatur su...
1,1,هديل الزهر,ريف دمشق - دوما,دمشق,"[Civil, Criminal, Arbitration, Legal]",5yearyear,قلمكم موقع متخصص بتقديم خدمة الترجمة المعتمدة،...,20,2star,Sunt vel eum at praesentium esse. Laudantium d...
2,1,هديل الزهر,ريف دمشق - دوما,دمشق,"[Civil, Criminal, Arbitration, Legal]",5yearyear,قلمكم موقع متخصص بتقديم خدمة الترجمة المعتمدة،...,24,2star,Reprehenderit sint minima voluptas. Sed labori...
3,2,عبدالرزاق شاكر اسيد الخضيري,79 ممر ياسمين السماعيل عمارة رقم 42\nبيشة,جيزان,"[Military, Arbitration, Commercial]",17yearyear,Qui velit dolor consequatur. Optio facilis tem...,33,3star,Ut dignissimos molestiae quos praesentium. Qui...
4,2,عبدالرزاق شاكر اسيد الخضيري,79 ممر ياسمين السماعيل عمارة رقم 42\nبيشة,جيزان,"[Military, Arbitration, Commercial]",17yearyear,Qui velit dolor consequatur. Optio facilis tem...,39,3star,Totam culpa fugit dicta error est. Consequatur...


In [52]:
lawyers_with_rates= pd.merge(agencies,lawyers_with_rates,on=["lawyer_id"],how="inner")
lawyers_with_rates.head()

,agency_id,lawyer_id,name,address,union_branch,specializations,years_of_experience,description,user_id,rating,review
0,1,1,هديل الزهر,ريف دمشق - دوما,دمشق,"[Civil, Criminal, Arbitration, Legal]",5yearyear,قلمكم موقع متخصص بتقديم خدمة الترجمة المعتمدة،...,3,5star,Inventore harum beatae aut esse consequatur su...
1,1,1,هديل الزهر,ريف دمشق - دوما,دمشق,"[Civil, Criminal, Arbitration, Legal]",5yearyear,قلمكم موقع متخصص بتقديم خدمة الترجمة المعتمدة،...,20,2star,Sunt vel eum at praesentium esse. Laudantium d...
2,1,1,هديل الزهر,ريف دمشق - دوما,دمشق,"[Civil, Criminal, Arbitration, Legal]",5yearyear,قلمكم موقع متخصص بتقديم خدمة الترجمة المعتمدة،...,24,2star,Reprehenderit sint minima voluptas. Sed labori...
3,2,21,ريما المشيقح,47 ممر ذيب الشهيل\nالقريات,الرياض,"[Legal, Criminal]",36yearyear,Aut recusandae vel veniam et sunt iusto. Id ra...,32,5star,Voluptates quisquam officia pariatur non. Quas...
4,2,21,ريما المشيقح,47 ممر ذيب الشهيل\nالقريات,الرياض,"[Legal, Criminal]",36yearyear,Aut recusandae vel veniam et sunt iusto. Id ra...,34,5star,Omnis consectetur ut exercitationem. Autem et ...


In [53]:
lawyers_with_rates = pd.merge(issues,lawyers_with_rates,on="agency_id",how="inner")
lawyers_with_rates.head()

,id,agency_id,start_date,end_date,status,estimated_cost,is_active,lawyer_id,name,address,union_branch,specializations,years_of_experience,description,user_id,rating,review
0,1,1,2024-10-20,None,قيد الدعوى,1500000,Yes,1,هديل الزهر,ريف دمشق - دوما,دمشق,"[Civil, Criminal, Arbitration, Legal]",5yearyear,قلمكم موقع متخصص بتقديم خدمة الترجمة المعتمدة،...,3,5star,Inventore harum beatae aut esse consequatur su...
1,1,1,2024-10-20,None,قيد الدعوى,1500000,Yes,1,هديل الزهر,ريف دمشق - دوما,دمشق,"[Civil, Criminal, Arbitration, Legal]",5yearyear,قلمكم موقع متخصص بتقديم خدمة الترجمة المعتمدة،...,20,2star,Sunt vel eum at praesentium esse. Laudantium d...
2,1,1,2024-10-20,None,قيد الدعوى,1500000,Yes,1,هديل الزهر,ريف دمشق - دوما,دمشق,"[Civil, Criminal, Arbitration, Legal]",5yearyear,قلمكم موقع متخصص بتقديم خدمة الترجمة المعتمدة،...,24,2star,Reprehenderit sint minima voluptas. Sed labori...
3,2,85,1990-01-13,2025-01-09,Atque beatae aut illo.,11672000,No,42,معاذ الحسين,9304 ممر يحيى جواهرجي\nالباحة,الباحة,"[Criminal, Administrative, Arbitration]",14yearyear,Tempora ut voluptas vel perspiciatis incidunt ...,12,4star,Consectetur non accusantium veritatis quia. Ma...
4,2,85,1990-01-13,2025-01-09,Atque beatae aut illo.,11672000,No,42,معاذ الحسين,9304 ممر يحيى جواهرجي\nالباحة,الباحة,"[Criminal, Administrative, Arbitration]",14yearyear,Tempora ut voluptas vel perspiciatis incidunt ...,4,4star,Est neque doloribus modi. Non rerum et cum max...


In [55]:
lawyers_with_rates["Text"] = lawyers_with_rates[['status','name','address','union_branch','years_of_experience','rating','estimated_cost',"review"]].astype(str).agg(" ".join,axis=1)
lawyers_with_rates.head()

,id,agency_id,start_date,end_date,status,estimated_cost,is_active,lawyer_id,name,address,union_branch,specializations,years_of_experience,description,user_id,rating,review,Text
0,1,1,2024-10-20,None,قيد الدعوى,1500000,Yes,1,هديل الزهر,ريف دمشق - دوما,دمشق,"[Civil, Criminal, Arbitration, Legal]",5yearyear,قلمكم موقع متخصص بتقديم خدمة الترجمة المعتمدة،...,3,5star,Inventore harum beatae aut esse consequatur su...,قيد الدعوى هديل الزهر ريف دمشق - دوما دمشق 5ye...
1,1,1,2024-10-20,None,قيد الدعوى,1500000,Yes,1,هديل الزهر,ريف دمشق - دوما,دمشق,"[Civil, Criminal, Arbitration, Legal]",5yearyear,قلمكم موقع متخصص بتقديم خدمة الترجمة المعتمدة،...,20,2star,Sunt vel eum at praesentium esse. Laudantium d...,قيد الدعوى هديل الزهر ريف دمشق - دوما دمشق 5ye...
2,1,1,2024-10-20,None,قيد الدعوى,1500000,Yes,1,هديل الزهر,ريف دمشق - دوما,دمشق,"[Civil, Criminal, Arbitration, Legal]",5yearyear,قلمكم موقع متخصص بتقديم خدمة الترجمة المعتمدة،...,24,2star,Reprehenderit sint minima voluptas. Sed labori...,قيد الدعوى هديل الزهر ريف دمشق - دوما دمشق 5ye...
3,2,85,1990-01-13,2025-01-09,Atque beatae aut illo.,11672000,No,42,معاذ الحسين,9304 ممر يحيى جواهرجي\nالباحة,الباحة,"[Criminal, Administrative, Arbitration]",14yearyear,Tempora ut voluptas vel perspiciatis incidunt ...,12,4star,Consectetur non accusantium veritatis quia. Ma...,Atque beatae aut illo. معاذ الحسين 9304 ممر يح...
4,2,85,1990-01-13,2025-01-09,Atque beatae aut illo.,11672000,No,42,معاذ الحسين,9304 ممر يحيى جواهرجي\nالباحة,الباحة,"[Criminal, Administrative, Arbitration]",14yearyear,Tempora ut voluptas vel perspiciatis incidunt ...,4,4star,Est neque doloribus modi. Non rerum et cum max...,Atque beatae aut illo. معاذ الحسين 9304 ممر يح...


In [61]:
lawyers_with_rates['processed_text'] = lawyers_with_rates['Text'].apply(split_by_language)
lawyers_with_rates.head()

,id,agency_id,start_date,end_date,status,estimated_cost,is_active,lawyer_id,name,address,union_branch,specializations,years_of_experience,description,user_id,rating,review,Text,processed_text
0,1,1,2024-10-20,None,قيد الدعوى,1500000,Yes,1,هديل الزهر,ريف دمشق - دوما,دمشق,"[Civil, Criminal, Arbitration, Legal]",5yearyear,قلمكم موقع متخصص بتقديم خدمة الترجمة المعتمدة،...,3,5star,Inventore harum beatae aut esse consequatur su...,قيد الدعوى هديل الزهر ريف دمشق - دوما دمشق 5ye...,"[الزهر, دمشق, قىد, الدعوى, هدىل, رىف, دوما, 5y..."
1,1,1,2024-10-20,None,قيد الدعوى,1500000,Yes,1,هديل الزهر,ريف دمشق - دوما,دمشق,"[Civil, Criminal, Arbitration, Legal]",5yearyear,قلمكم موقع متخصص بتقديم خدمة الترجمة المعتمدة،...,20,2star,Sunt vel eum at praesentium esse. Laudantium d...,قيد الدعوى هديل الزهر ريف دمشق - دوما دمشق 5ye...,"[الزهر, دمشق, قىد, الدعوى, هدىل, رىف, دوما, 5y..."
2,1,1,2024-10-20,None,قيد الدعوى,1500000,Yes,1,هديل الزهر,ريف دمشق - دوما,دمشق,"[Civil, Criminal, Arbitration, Legal]",5yearyear,قلمكم موقع متخصص بتقديم خدمة الترجمة المعتمدة،...,24,2star,Reprehenderit sint minima voluptas. Sed labori...,قيد الدعوى هديل الزهر ريف دمشق - دوما دمشق 5ye...,"[الزهر, دمشق, قىد, الدعوى, هدىل, رىف, دوما, no..."
3,2,85,1990-01-13,2025-01-09,Atque beatae aut illo.,11672000,No,42,معاذ الحسين,9304 ممر يحيى جواهرجي\nالباحة,الباحة,"[Criminal, Administrative, Arbitration]",14yearyear,Tempora ut voluptas vel perspiciatis incidunt ...,12,4star,Consectetur non accusantium veritatis quia. Ma...,Atque beatae aut illo. معاذ الحسين 9304 ممر يح...,"[الباحه, ممر, الحسىن, جواهرجى, ىحىى, consectet..."
4,2,85,1990-01-13,2025-01-09,Atque beatae aut illo.,11672000,No,42,معاذ الحسين,9304 ممر يحيى جواهرجي\nالباحة,الباحة,"[Criminal, Administrative, Arbitration]",14yearyear,Tempora ut voluptas vel perspiciatis incidunt ...,4,4star,Est neque doloribus modi. Non rerum et cum max...,Atque beatae aut illo. معاذ الحسين 9304 ممر يح...,"[الباحه, ممر, الحسىن, جواهرجى, ىحىى, doloribu,..."


In [64]:
documents = lawyers_with_rates[['lawyer_id','processed_text']].copy()
documents['processed_text'] = documents['processed_text'].apply(lambda x:" ".join(x))
documents.rename(columns={
    "lawyer_id":"docno",
    "processed_text":"text"
},inplace=True)
documents.head()

,docno,text
0,1,الزهر دمشق قىد الدعوى هدىل رىف دوما 5yearyear ...
1,1,الزهر دمشق قىد الدعوى هدىل رىف دوما 5yearyear ...
2,1,الزهر دمشق قىد الدعوى هدىل رىف دوما non 5yeary...
3,42,الباحه ممر الحسىن جواهرجى ىحىى consectetur dol...
4,42,الباحه ممر الحسىن جواهرجى ىحىى doloribu maxim ...


In [65]:
index_path = r'C:\Users\Mohammad Kher\Desktop\Projects\IR\lawyer_index'
indexer = pt.DFIndexer(index_path,overwrite=True)

index_ref = indexer.index(documents['text'],documents['docno'].astype(str))

print("Index created with reference:", index_ref)

C:\Users\Mohammad Kher\AppData\Local\Temp\ipykernel_6428\485533886.py:2: DeprecationWarning: Call to deprecated class DFIndexer. (use pt.terrier.IterDictIndexer().index(dataframe.to_dict(orient='records')) instead) -- Deprecated since version 0.11.0.
  indexer = pt.DFIndexer(index_path,overwrite=True)


18:42:15.802 [main] ERROR org.terrier.structures.indexing.Indexer -- Could not rename index
java.io.IOException: Rename of index structure file 'C:\Users\Mohammad Kher\Desktop\Projects\IR\lawyer_index/data_1.direct.bf' (exists) to 'C:\Users\Mohammad Kher\Desktop\Projects\IR\lawyer_index/data.direct.bf' (exists) failed - likely that source file is still open. Possible indexing bug?
	at org.terrier.structures.IndexUtil.renameIndex(IndexUtil.java:379)
	at org.terrier.structures.indexing.Indexer.index(Indexer.java:388)
	at org.terrier.structures.indexing.Indexer.index(Indexer.java:355)
Index created with reference: <org.terrier.querying.IndexRef at 0x20b7ffeba70 jclass=org/terrier/querying/IndexRef jself=<LocalRef obj=0x757067b2 at 0x20b7fc11c30>>


In [66]:
index = pt.IndexFactory.of(index_ref)
print(index.getCollectionStatistics().toString())

Number of documents: 20
Number of terms: 55
Number of postings: 208
Number of fields: 0
Number of tokens: 208
Field names: []
Positions:   false



In [67]:
# bm25 = pt.BatchRetrieve(index_ref, wmodel="BM25")
# query = "experienced lawyer"
# results = bm25.search(query)
# print(results[['docno', 'score']])

In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np 

In [69]:
user_input = "experienced lawyer"

In [70]:
all_descriptions = [user_input] + documents["text"].tolist()
all_descriptions

['experienced lawyer',
 'الزهر دمشق قىد الدعوى هدىل رىف دوما 5yearyear inventor consequatur 1500000 fugit sunt quia sit eum aut temporibu beata dolorem harum ess 5star',
 'الزهر دمشق قىد الدعوى هدىل رىف دوما 5yearyear praesentium 1500000 sunt omni dolor vel eum laudantium delen 2star ess',
 'الزهر دمشق قىد الدعوى هدىل رىف دوما non 5yearyear volupta 1500000 fugit sunt sed minima sint laboriosam incidunt fugiat 2star quia sit velit reprehenderit',
 'الباحه ممر الحسىن جواهرجى ىحىى consectetur dolor quia sit id 14yearyear illo accusantium et 11672000 non magnam veritati aut ut nequ atqu 4star est 9304 beata',
 'الباحه ممر الحسىن جواهرجى ىحىى doloribu maxim provid modi 14yearyear illo fugiat dolor deserunt et cum 11672000 non nulla rerum aut nequ atqu 4star est qui 9304 beata',
 'الباحه ممر الحسىن جواهرجى ىحىى volupta officia rem vita error sunt 14yearyear illo 2star et ess natu voluptatem 11672000 aut atqu sed delectu 9304 beata ad',
 'الباحه ممر الحسىن جواهرجى ىحىى 11672000 consequatur fu

In [71]:
vectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix = vectorizer.fit_transform(all_descriptions)

In [72]:
user_vector = tfidf_matrix[0]  
lawyer_vectors = tfidf_matrix[1:]


In [73]:
similarities = cosine_similarity(user_vector, lawyer_vectors).flatten()

In [74]:
ranked_lawyers = np.argsort(similarities)[::-1]

In [75]:
print("\nTop Recommendations:")
for idx in ranked_lawyers:
    lawyer_id = documents.iloc[idx]["docno"]
    description = documents.iloc[idx]["text"]
    similarity = similarities[idx]
    print(f"Lawyer ID: {lawyer_id}, Similarity {similarity}, Description: {description}")


Top Recommendations:
Lawyer ID: 1, Similarity 0.0, Description: الزهر دمشق قىد الدعوى هدىل رىف دوما 5yearyear inventor consequatur 1500000 fugit sunt quia sit eum aut temporibu beata dolorem harum ess 5star
Lawyer ID: 45, Similarity 0.0, Description: السىف شقه شرحبىل الباطن طرىق الماجد حفر رقم عرىن aspernatur assumenda 25yearyear reiciendi sunt 1star quod et molestia harum ipsum non 45 22631383 culpa ut 26 nemo velit nequ ab quo
Lawyer ID: 45, Similarity 0.0, Description: السىف شقه شرحبىل الباطن طرىق الماجد حفر رقم عرىن volupta 25yearyear alia sit sequi id illo et harum non 45 22631383 nulla dicta quae ut 26 nemo nequ quaerat ab beata 3star aspernatur totam
Lawyer ID: 45, Similarity 0.0, Description: السىف شقه شرحبىل الباطن طرىق الماجد حفر رقم عرىن 45 corpori reprehenderit 25yearyear 22631383 sint ab 2star eveniet maxim 26 nemo aspernatur cum dolor et
Lawyer ID: 45, Similarity 0.0, Description: السىف شقه شرحبىل الباطن طرىق الماجد حفر رقم عرىن 25yearyear eum temporibu modi quibusdam et